<a href="https://colab.research.google.com/github/anavaz03/git-workshop/blob/main/Dados_externos_ficheiros_locais%2C_Drive%2C_Sheets_e_Cloud_Storage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este bloco de notas fornece receitas para carregar e guardar dados de fontes externas.

# Sistema de ficheiros local

## Carregar ficheiros a partir do seu sistema de ficheiros local

<code>files.upload</code> devolve um dicionário dos ficheiros que foram carregados.
O dicionário é indexado com o nome do ficheiro e os valores são os dados que foram carregados.

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

## Transferir ficheiros para o seu sistema de ficheiros local

<code>files.download</code> irá invocar a transferência do ficheiro do navegador para o seu computador local.


In [ ]:
from google.colab import files

with open('example.txt', 'w') as f:
  f.write('some content')

files.download('example.txt')

# Google Drive

Pode aceder aos ficheiros no Drive de várias formas, incluindo:
- Montar o Google Drive na máquina virtual do tempo de execução
- Usar um wrapper à volta da API, tal como <a href="https://docs.iterative.ai/PyDrive2/">PyDrive2</a>
- Usar a <a href="https://developers.google.com/drive/v3/web/about-sdk">API REST nativa</a>



Seguem-se alguns exemplos de cada um deles:

## Montar o Google Drive localmente

O exemplo abaixo mostra como montar o Google Drive no seu tempo de execução através de um código de autorização e como escrever e ler ficheiros no mesmo. Após a execução, pode ver o novo ficheiro &#40;<code>foo.txt</code>&#41; em <a href="https://drive.google.com/">https://drive.google.com/</a>.

Isto só suporta ler, escrever e mover ficheiros. Para modificar as definições de partilha ou os outros metadados de forma programática, utilize uma das outras opções abaixo.

<strong>Nota:</strong> ao utilizar o botão "Montar Drive" no navegador de ficheiros, não são necessários códigos de autenticação para blocos de notas que só foram editados pelo utilizador atual.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code
Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
with open('/content/drive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /content/drive/My\ Drive/foo.txt

Hello Google Drive!

In [ ]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.


## PyDrive2

Os exemplos abaixo demonstram a autenticação e o carregamento/transferência de ficheiros através do PyDrive2. Estão disponíveis mais exemplos na <a href="https://docs.iterative.ai/PyDrive2/">documentação do PyDrive2</a>.

In [ ]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Autentique e crie o cliente do PyDrive2.


In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Crie e carregue um ficheiro de texto.


In [ ]:
uploaded = drive.CreateFile({'title': 'Sample upload.txt'})
uploaded.SetContentString('Sample upload file content')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 14vDAdqp7BSCQnoougmgylBexIr2AQx2T


Carregue um ficheiro por ID e imprima o respetivo conteúdo.


In [ ]:
downloaded = drive.CreateFile({'id': uploaded.get('id')})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

Downloaded content "Sample upload file content"


## API REST do Drive

Para utilizar a API Drive, primeiro temos de autenticar e criar um cliente API.


In [ ]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

Com este cliente, podemos utilizar qualquer uma das funções da <a href="https://developers.google.com/drive/v3/reference/">referência da API Google Drive</a>. Seguem-se alguns exemplos.


### Criar um novo ficheiro do Drive com dados a partir do Python

Primeiro, crie um ficheiro local para carregar.

In [ ]:
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

Carregue com o método <a href="https://developers.google.com/drive/v3/reference/files/create"><code>files.create</code></a>. Estão disponíveis mais detalhes sobre o carregamento de ficheiros na <a href="https://developers.google.com/drive/v3/web/manage-uploads">documentação do programador</a>.

In [ ]:
from googleapiclient.http import MediaFileUpload

file_metadata = {
  'name': 'Sample file',
  'mimeType': 'text/plain'
}
media = MediaFileUpload('/tmp/to_upload.txt',
                        mimetype='text/plain',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id')))

File ID: 1Cw9CqiyU6zbXFD9ViPZu_3yX-sYF4W17


Após executar a célula acima, é apresentado um novo ficheiro chamado "Sample file" em <a href="https://drive.google.com/">https://drive.google.com/</a>.

### Transferir dados de um ficheiro do Drive para o Python

Transfira o ficheiro que carregámos acima.

In [ ]:
file_id = created.get('id')

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

downloaded.seek(0)
print('Downloaded file contents are: {}'.format(downloaded.read()))

Downloaded file contents are: b'my sample file'


Para transferir um ficheiro diferente, defina <code>file&#95;id</code> acima para o ID desse ficheiro, que será algo como: "1uBtlaggVyWshwcyP6kEI-y&#95;W3P8D26sz".

# Google Sheets


## Extensão do Google Sheets do Workspace

Temos uma extensão do Workspace, o <a href="https://workspace.google.com/u/0/marketplace/app/sheets_to_colab/945625412720">Sheets para o Colab</a>, que lhe permite importar diretamente dados do Google Sheets para o Colab a partir da IU do Sheets. Siga o link para a extensão Sheets para o Colab do Workspace para saber mais.

## Interação com o Google Sheets através do gspread

Também pode usar a biblioteca do <a href="https://github.com/burnash/gspread"><code>gspread</code></a> de código aberto para interagir com o Google Sheets. O código abaixo mostra como configurar e autenticar o <code>gspread</code>.

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

Abaixo encontra-se um pequeno conjunto de exemplos de <code>gspread</code>. Estão disponíveis exemplos adicionais na <a href="https://github.com/burnash/gspread#more-examples">página sobre <code>gspread</code> do GitHub</a>.

### Criar uma nova página com dados a partir do Python

In [ ]:
sh = gc.create('My cool spreadsheet')

Após executar a célula acima, é apresentada uma nova folha de cálculo chamada "My cool spreadsheet" em <a href="https://sheets.google.com/">https://sheets.google.com</a>.

Abra a nova folha e adicione alguns dados aleatórios.

In [ ]:
worksheet = gc.open('My cool spreadsheet').sheet1

cell_list = worksheet.range('A1:C2')

import random
for cell in cell_list:
  cell.value = random.randint(1, 10)

worksheet.update_cells(cell_list)

{'spreadsheetId': '1dsQeN0YzXuM387l_CuyEbsYzL2ew9TJFzR-E-RQnwxs',
 'updatedCells': 6,
 'updatedColumns': 3,
 'updatedRange': 'Sheet1!A1:C2',
 'updatedRows': 2}

### Transferir dados de uma folha para o Python como um Pandas DataFrame

Leia novamente os dados aleatórios que inserimos acima e converta o resultado num <a href="https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html">Pandas DataFrame</a>.

In [ ]:
worksheet = gc.open('My cool spreadsheet').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

import pandas as pd
pd.DataFrame.from_records(rows)

[['6', '3', '4'], ['7', '2', '1']]


,0,1,2
0,6,3,4
1,7,2,1


# InteractiveSheet

Agora, pode incorporar folhas de cálculo do Google Sheets publicadas no Colab com a biblioteca <code>InteractiveSheet</code>. Isto significa que pode criar e editar dados no Google Sheets e incorporá-los perfeitamente no seu bloco de notas com os DataFrames do Pandas, tudo a partir do Colab.

In [ ]:
from google.colab import sheets

# Create a new interactive sheet and add data to it.
sheet = sheets.InteractiveSheet()

In [ ]:
# Get a Pandas DataFrame from the selected worksheet
df = sheet.as_df()

In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame(np.random.randn(10, 4), columns=list('ABCD'))

# Create a new sheet and include the column names as the first row.
sheet = sheets.InteractiveSheet(df=df, title='foo', include_column_headers=True)

In [ ]:
# Push data from Colab to the selected worksheet
df2 = pd.DataFrame(np.random.randn(10, 4), columns=list('ABCD'))
sheet.update(df=df)

In [ ]:
# Display the sheet in the output of the current cell
sheet.display()

# Google Cloud Storage &#40;GCS&#41;

Para utilizar o Colaboratory com o GCS, tem de criar um <a href="https://cloud.google.com/storage/docs/projects">projeto do Google Cloud</a> ou utilizar um projeto pré-existente.

Especifique o ID do seu projeto abaixo:

In [ ]:
project_id = 'Your_project_ID_here'

Os ficheiros no GCS estão contidos em <a href="https://cloud.google.com/storage/docs/buckets">contentores</a>.

Os contentores têm de ter um nome exclusivo a nível global, por isso geramos um aqui.

In [ ]:
import uuid
bucket_name = 'colab-sample-bucket-' + str(uuid.uuid1())

Para aceder ao GCS, temos de autenticar.

In [ ]:
from google.colab import auth
auth.authenticate_user()

O GCS pode ser acedido através do utilitário da linha de comandos <code>gsutil</code> ou da API nativa do Python.

## `gsutil`

Primeiro, configuramos <code>gsutil</code> para utilizar o projeto que especificámos acima ao utilizar <code>gcloud</code>.

In [ ]:
!gcloud config set project {project_id}

Updated property [core/project].


Crie um ficheiro local para carregar.

In [ ]:
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

Crie um contentor para o qual iremos carregar o ficheiro &#40;<a href="https://cloud.google.com/storage/docs/gsutil/commands/mb">documentação</a>&#41;.

In [ ]:
!gsutil mb gs://{bucket_name}

Creating gs://colab-sample-bucket-44971372-baaf-11e7-ae30-0242ac110002/...


Copie o ficheiro para o novo contentor &#40;<a href="https://cloud.google.com/storage/docs/gsutil/commands/cp">documentação</a>&#41;.

In [ ]:
!gsutil cp /tmp/to_upload.txt gs://{bucket_name}/

Copying file:///tmp/to_upload.txt [Content-Type=text/plain]...
/ [1 files][   14.0 B/   14.0 B]                                                
Operation completed over 1 objects/14.0 B.                                       


Despeje o conteúdo do ficheiro copiado recentemente para se certificar de que tudo funcionou &#40;<a href="https://cloud.google.com/storage/docs/gsutil/commands/cat">documentação</a>&#41;.


In [ ]:
!gsutil cat gs://{bucket_name}/to_upload.txt

my sample file

In [ ]:
# @markdown Quando o carregamento estiver concluído, os dados são apresentados no navegador de armazenamento da Cloud Console do seu projeto:
print('https://console.cloud.google.com/storage/browser?project=' + project_id)

https://console.cloud.google.com/storage/browser?project=Your_project_ID_here


Por último, transferimos o ficheiro que acabámos de carregar no exemplo acima. É tão simples como inverter a ordem no comando <code>gsutil cp</code>.

In [ ]:
!gsutil cp gs://{bucket_name}/to_upload.txt /tmp/gsutil_download.txt

# Imprima o resultado para se certificar de que a transferência funcionou.
!cat /tmp/gsutil_download.txt

Copying gs://colab-sample-bucket483f20dc-baaf-11e7-ae30-0242ac110002/to_upload.txt...
/ [1 files][   14.0 B/   14.0 B]                                                
Operation completed over 1 objects/14.0 B.                                       
my sample file

## API do Python

Estes fragmentos baseiam-se num <a href="https://github.com/GoogleCloudPlatform/storage-file-transfer-json-python/blob/master/chunked_transfer.py">exemplo maior</a> que mostra as utilizações adicionais da API.

Primeiro, criamos o cliente do serviço.

In [ ]:
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')

Crie um ficheiro local para carregar.

In [ ]:
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

Crie um contentor no projeto especificado acima.

In [ ]:
# Utilize um nome de contentor exclusivo a nível global diferente do exemplo gsutil acima.
import uuid
bucket_name = 'colab-sample-bucket-' + str(uuid.uuid1())

body = {
  'name': bucket_name,
  # For a full list of locations, see:
  # https://cloud.google.com/storage/docs/bucket-locations
  'location': 'us',
}
gcs_service.buckets().insert(project=project_id, body=body).execute()
print('Done')

Done


Carregue o ficheiro para o contentor criado recentemente.

In [ ]:
from googleapiclient.http import MediaFileUpload

media = MediaFileUpload('/tmp/to_upload.txt',
                        mimetype='text/plain',
                        resumable=True)

request = gcs_service.objects().insert(bucket=bucket_name,
                                       name='to_upload.txt',
                                       media_body=media)

response = None
while response is None:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, response = request.next_chunk()

print('Upload complete')

Upload complete


In [ ]:
# @markdown Quando o carregamento estiver concluído, os dados são apresentados no navegador de armazenamento da Cloud Console do seu projeto:
print('https://console.cloud.google.com/storage/browser?project=' + project_id)

https://console.cloud.google.com/storage/browser?project=Your_project_ID_here


Transfira o ficheiro que acabámos de carregar.

In [ ]:
from apiclient.http import MediaIoBaseDownload

with open('/tmp/downloaded_from_gcs.txt', 'wb') as f:
  request = gcs_service.objects().get_media(bucket=bucket_name,
                                            object='to_upload.txt')
  media = MediaIoBaseDownload(f, request)

  done = False
  while not done:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, done = media.next_chunk()

print('Download complete')

Download complete


Inspecione o ficheiro transferido.


In [ ]:
!cat /tmp/downloaded_from_gcs.txt

my sample file